In [2]:
import numpy as np
import pandas as pd
import regex as re


In [3]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm, tqdm_notebook

In [4]:
nltk.download('stopwords')
# Downloading wordnet before applying Lemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Suswarah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Suswarah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Suswarah\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [19]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
def import_data(file_path):
    return pd.read_csv(file_path)


In [7]:
def handle_nulls(df):
    df.replace("", pd.NA, inplace = True)
    df.replace(" ", pd.NA, inplace = True)
    df.dropna(subset=['Review text', 'Review Title'], how='all', inplace=True)
    df['Review Title'].replace(pd.NA, "None", inplace = True)
    df.isnull().sum()
    return df
    

In [8]:
def feature_extraction(df):
    df.drop(['Reviewer Name', 'Place of Review', 'Up Votes', 'Down Votes', 'Month'], axis=1, inplace = True)
    return df

In [9]:
def feature_engineering(df):
    df['Review text'] = df['Review text'].str.replace(r'READ MORE', '', regex=True)
    # Use replace() with if condition to create the target variable 'Sentiment'
    df['Sentiment'] = df['Ratings'].replace({rating: 1 if rating >= 3 else 0 for rating in df['Ratings']})
    df["Review"] = df['Review Title'] + " " + df['Review text']
    df.drop(['Review Title', 'Review text', 'Ratings'], axis = 1, inplace = True)
    df.head()
    return df

In [10]:
def split_inputs_output(data, inputs, output):
    
    x = data[inputs]
    y = data[output]
    return x, y

In [11]:
def split_train_test(x, y, test_size=0.25, random_state=0):
    
    return train_test_split(x, y, test_size=test_size, random_state=random_state)


In [12]:
def preprocessor(text):
    # Removing special characters and digits
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    # change sentence to lower case
    letters_only = letters_only.lower()
    # tokenize into words
    words = letters_only.split()
    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)


In [13]:
def preprocess_data(X_train, X_test, y_train, y_test):
    # Define TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessor)

    # Fit TF-IDF vectorizer on X_train and transform X_train and X_test
    x_train_transformed = tfidf_vectorizer.fit_transform(X_train)
    x_test_transformed = tfidf_vectorizer.transform(X_test)
 
    return x_train_transformed, x_test_transformed, y_train, y_test



In [14]:
def train_model(x_train_transformed, y_train, hyperparameters):

    clf = LogisticRegression(**hyperparameters)
    clf.fit(x_train_transformed, y_train)
    return clf


In [15]:
def evaluate_model(model, x_train_transformed, y_train, x_test_transformed, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(x_train_transformed)
    y_test_pred = model.predict(x_test_transformed)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

**Best Model**: Logistic Regression

**Parameters**:
        
        'vectorization__max_features': 2000,
        'classifier__C': 10,
        'classifier__penalty': l2
        
**Accuracy**: 92%

**F1 - score Positive Review Prediction** : 96%

**F1 - score Negative Review Prediction** : 62%

In [29]:
def workflow(data_path):
    DATA_PATH = data_path
    INPUTS = 'Review'
    OUTPUT = 'Sentiment'
    HYPERPARAMETERS = {'C': 10, 'max_iter': 5000, 'penalty': 'l2'}


    # Load data
    df = import_data(DATA_PATH)
    
    # Handle nulls
    df = handle_nulls(df)
    
    # Extract Features
    df = feature_extraction(df)
    
    # Feature Engineering
    df = feature_engineering(df)
    print("Nulls in data:\n",df.isnull().sum())

    
    # Identify Inputs and Output
    x, y = split_inputs_output(df, INPUTS, OUTPUT)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = split_train_test(x, y)
    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
    
    # Preprocess the data
    x_train_transformed, x_test_transformed, y_train, y_test = preprocess_data(x_train, x_test, y_train, y_test)

    # Build a model
    model = train_model(x_train_transformed, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, x_train_transformed, y_train, x_test_transformed, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

In [30]:
if __name__ == "__main__":
    workflow(data_path="C:\\Users\\Suswarah\\Downloads\\MLOps\\MLFlow\\badminton_review_data.csv")


Nulls in data:
 Sentiment    0
Review       0
dtype: int64
(7,) (3,) (7,) (3,)
Train Score: 1.0
Test Score: 0.3333333333333333


# Prefect

In [43]:
#pip install prefect

In [33]:
from prefect import task, flow

In [44]:
@task
def import_data(file_path):
    return pd.read_csv(file_path)


@task
def handle_nulls(df):
    df.replace("", pd.NA, inplace = True)
    df.replace(" ", pd.NA, inplace = True)
    df.dropna(subset=['Review text', 'Review Title'], how='all', inplace=True)
    df['Review Title'].replace(pd.NA, "None", inplace = True)
    df.isnull().sum()
    return df
    


@task
def feature_extraction(df):
    df.drop(['Reviewer Name', 'Place of Review', 'Up Votes', 'Down Votes', 'Month'], axis=1, inplace = True)
    return df


@task
def feature_engineering(df):
    df['Review text'] = df['Review text'].str.replace(r'READ MORE', '', regex=True)
    # Use replace() with if condition to create the target variable 'Sentiment'
    df['Sentiment'] = df['Ratings'].replace({rating: 1 if rating >= 3 else 0 for rating in df['Ratings']})
    df["Review"] = df['Review Title'] + " " + df['Review text']
    df.drop(['Review Title', 'Review text', 'Ratings'], axis = 1, inplace = True)
    df.head()
    return df


@task
def split_inputs_output(data, inputs, output):
    
    x = data[inputs]
    y = data[output]
    return x, y


@task
def split_train_test(x, y, test_size=0.25, random_state=0):
    
    return train_test_split(x, y, test_size=test_size, random_state=random_state)




@task
def preprocess_data(X_train, X_test, y_train, y_test):
    def preprocessor(text):
        # Removing special characters and digits
        letters_only = re.sub("[^a-zA-Z]", " ", text)
        # change sentence to lower case
        letters_only = letters_only.lower()
        # tokenize into words
        words = letters_only.split()
        # remove stop words
        words = [word for word in words if word not in stopwords.words("english")]
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]
        return " ".join(words)

    # Define TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessor)

    # Fit TF-IDF vectorizer on X_train and transform X_train and X_test
    x_train_transformed = tfidf_vectorizer.fit_transform(X_train)
    x_test_transformed = tfidf_vectorizer.transform(X_test)
    return x_train_transformed, x_test_transformed, y_train, y_test


@task
def train_model(x_train_transformed, y_train, hyperparameters):

    clf = LogisticRegression(**hyperparameters)
    clf.fit(x_train_transformed, y_train)
    return clf



@task
def evaluate_model(model, x_train_transformed, y_train, x_test_transformed, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(x_train_transformed)
    y_test_pred = model.predict(x_test_transformed)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score





In [45]:
@flow(name="Logistic Regression final Flow")
def workflow():
    DATA_PATH = "C:\\Users\\Suswarah\\Downloads\\MLOps\\MLFlow\\badminton_review_data.csv"
    INPUTS = 'Review'
    OUTPUT = 'Sentiment'
    HYPERPARAMETERS = {'C': 10, 'max_iter': 5000, 'penalty': 'l2'}


    # Load data
    df = import_data(DATA_PATH)
    
    # Handle nulls
    df = handle_nulls(df)
    
    # Extract Features
    df = feature_extraction(df)
    
    # Feature Engineering
    df = feature_engineering(df)

    
    # Identify Inputs and Output
    x, y = split_inputs_output(df, INPUTS, OUTPUT)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = split_train_test(x, y)
    
    # Preprocess the data
    x_train_transformed, x_test_transformed, y_train, y_test = preprocess_data(x_train, x_test, y_train, y_test)

    # Build a model
    model = train_model(x_train_transformed, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, x_train_transformed, y_train, x_test_transformed, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)



In [46]:
if __name__ == "__main__":
    workflow()

19:33:45.701 | INFO    | prefect.engine - Created flow run 'futuristic-shark' for flow 'Logistic Regression final Flow'

19:33:45.983 | INFO    | Flow run 'futuristic-shark' - Created task run 'import_data-0' for task 'import_data'

19:33:45.983 | INFO    | Flow run 'futuristic-shark' - Executing 'import_data-0' immediately...

19:33:46.374 | INFO    | Task run 'import_data-0' - Finished in state Completed()

19:33:46.482 | INFO    | Flow run 'futuristic-shark' - Created task run 'handle_nulls-0' for task 'handle_nulls'

...

19:34:12.300 | INFO    | Flow run 'futuristic-shark' - Created task run 'evaluate_model-0' for task 'evaluate_model'

19:34:12.313 | INFO    | Flow run 'futuristic-shark' - Executing 'evaluate_model-0' immediately...

19:34:12.662 | INFO    | Task run 'evaluate_model-0' - Finished in state Completed()

Train Score: 0.9550297712315888
Test Score: 0.9144736842105263


19:34:12.813 | INFO    | Flow run 'futuristic-shark' - Finished in state Completed('All states completed.')

In [ ]:
# if __name__ == "__main__":
#     workflow.serve(
#         name="my-first-deployment",
#         cron="* * * * *"
#     )

In [49]:
%run FlipkartReviewCron.py


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Suswarah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Suswarah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Suswarah\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\Suswarah\anaconda3\Lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'import_data' and defined at 'C:\Users\Suswarah\Downloads\MLOps\Orchestrate_Models\FlipkartReviewCron.py:31' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
C:\Users\Suswarah\anaconda3\Lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'handle_nulls' and defined at 'C:\Users\Suswarah\Downloads\MLOps\Orc